<a href="https://colab.research.google.com/github/fnastasi/8648_Robotica_Movil_TP_Final/blob/main/Desafio_Patrullaje/Desafio_Patrullaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Librerías que uso
import matplotlib as mlp
%matplotlib notebook

import numpy as np
import matplotlib.image as pimg
# from enum import Enum
import math
import threading
import time
import timeit
import json

from math import *
import numpy.linalg as lng
import argparse
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import matplotlib.animation as animation
import matplotlib.patches as ppatches

import functools
import sys

import importlib

from scipy import signal

from diffRobot import DiffRobot

# funciones para hacer ploteos
from ploteo_de_la_simulacion import drawRobot
from ploteo_de_la_simulacion import drawRobotPos
from ploteo_de_la_simulacion import drawLidarMeasure
from ploteo_de_la_simulacion import updatedLidarDraw

# Creo una carpeta para guardar los gráficos
!mkdir plots

In [3]:
# clase particle filter + clase partículas

# =============================================================================
# Clase partícula
# =============================================================================
class particle():
    def __init__(self):
        self.x = np.random.uniform(-1.5,1)
        self.y = np.random.uniform(-1.5,1.5)
        self.theta = np.random.uniform(-np.pi,np.pi)
        
        self.peso = 1.0
    #========================================================================#
    def particle_pos(self):
        vector = np.array([self.x,self.y,self.theta])

        return vector
    #========================================================================#
    # modelo de movimiento de la partícula
    def mov_odom(self,v_0,w_0,t,alfa):

        v = v_0 + np.random.normal(0,alfa[0]*abs(v_0) + alfa[1]*abs(w_0))
        w = w_0 + np.random.normal(0,alfa[2]*abs(v_0) + alfa[3]*abs(w_0))

        theta_new = self.theta + w * t
        x_new = self.x + v * cos(theta_new) * t
        y_new = self.y + v * sin(theta_new) * t
        
        self.x = x_new
        self.y = y_new

        while(theta_new > np.pi):
          theta_new -= 2 * np.pi
        while(theta_new < -np.pi):
          theta_new += 2 * np.pi

        self.theta = theta_new

        result = particle()
        result.x = x_new
        result.y = y_new
        result.theta = theta_new

        return result
    #========================================================================#
    # setea la pose de la partícula
    def set_particle_pos(self,pose):
        self.x = pose[0]
        self.y = pose[1]
        self.theta = pose[2]
# =============================================================================
# Clase filtro de partículas
# =============================================================================
class particle_filter():
    def __init__(self,N,alfa,l,p):
        
        # inicialización del filtro de partículas. Se crean "N" partículas en
        # poses aleatorias
        self.noise_param = alfa
        
        self.lidar_resolution = l
        self.lidar_pos = p
        
        self.occupation_map_loaded = False
        
        self.particles = []
        self.pesos = []
        
        for i in range(0,N):
            # creo una partícula
            p = particle()

            self.particles.append(p)
            self.pesos.append(1.0)
        
        self.N = N
    #========================================================================#
    def entropia(self):
        H = 0
        for i in range(self.N):
            H -= np.log(self.particles[i].peso) * self.particles[i].peso
        
        return H
    #========================================================================#
    def mean_particle_pos(self):
        x_mean = 0
        y_mean = 0
        theta_mean = 0
        for i in range(self.N):
            x_mean = x_mean + self.particles[i].x
            y_mean = y_mean + self.particles[i].y
            theta_mean = theta_mean + self.particles[i].theta
        x_mean = x_mean / self.N
        y_mean = y_mean / self.N
        theta_mean = theta_mean / self.N
        
        pos_media = np.array([x_mean,y_mean,theta_mean])
        return pos_media
    #========================================================================#
    def best_particle_pos(self):

        peso_max = self.particles[0].peso
        best_particle = 0

        for i in range(1,self.N):
            peso_new = self.particles[i].peso
            if(peso_new > peso_max):
                peso_max = peso_new
                best_particle = i
        

        return best_particle
    #========================================================================#
    def std_err_particle_pos(self):
        # obengo la media
        mu = self.mean_particle_pos()

        dist = 0
        for i in range(self.N):

            dist = dist + (self.particles[i].x - mu[0])**2 + (self.particles[i].y - mu[1])**2
        

        desvio = np.sqrt(dist / self.N)
        

        return desvio
        
    #========================================================================#
    def loadoccmap(self,occupation_map,occupation_map_scale):
        self.occ_map = occupation_map
        self.occ_map_scale = occupation_map_scale
        self.occ_map_loaded = True
        self.occ_map_center = np.array(self.occ_map.shape) / 2.0  

    #========================================================================#
    def __calculateCellForPosition(self, x, y):
        norm_pos = np.array([x,y]) / self.occ_map_scale
        pos_occmap = norm_pos + self.occ_map_center

        if pos_occmap[1] > 0:
            row = int(self.occ_map.shape[0] - pos_occmap[1])
        else:
            row = self.occ_map.shape[0] + 1

        if pos_occmap[0] > 0:
            col = int(pos_occmap[0])
        else:
            col = -1
        return (row, col)
    #========================================================================#
    def __amIOutsideOccMap (self, x, y):
        out_of_map = True
        row = -1
        col = -1
        if self.occ_map_loaded == True:
            (row, col) = self.__calculateCellForPosition(x, y)
            
            if row < 0 or row >= self.occ_map.shape[0]:
                out_of_map = True
            elif col < 0 or col >= self.occ_map.shape[1]:
                out_of_map = True
            else:
                out_of_map = False 
        else:
            out_of_map = True
        
        return (out_of_map, row, col)
    #========================================================================#
    def __isItFreePosition(self, x, y):
        free_pos = False
        out_of_map = False
        if self.occ_map_loaded == True:
            (out_of_map, row, col) = self.__amIOutsideOccMap(x,y)
            if  out_of_map == False:
                occ_map_value = self.occ_map[row, col]
                if (occ_map_value < 0.5):
                    free_pos = False
                else:
                    free_pos = True
            else:
                free_pos = False
        else:
            free_pos = True

        return (free_pos, out_of_map)
    #========================================================================#
    def ray_casting(self, x, y, theta):
        # dx = math.cos(theta) *  self.occ_map_scale
        # dy = math.sin(theta) *  self.occ_map_scale
        dx = np.cos(theta) *  self.occ_map_scale
        dy = np.sin(theta) *  self.occ_map_scale
        #dx = np.cos(theta) *  0.
        #dy = np.sin(theta) *  0.1
        free_pos = True
        out_of_map = False

        next_x = x
        next_y = y
        last_free_pos_x = x
        last_free_pos_y = y
        while free_pos == True and out_of_map == False:
            last_free_pos_x = next_x
            last_free_pos_y = next_y
            next_x += dx
            next_y += dy
            (free_pos, out_of_map) = self.__isItFreePosition(next_x, next_y)
            #if free_pos == False or out_of_map == True:
            #    next_x = last_free_pos_x
            #    next_y = last_free_pos_y
            #    free_pos = True
            #    out_of_map = False
            #    ddx = dx / 8
            #    ddy = dy / 8
            #    while free_pos == True and out_of_map == False:
            #       last_free_pos_x = next_x
            #        last_free_pos_y = next_y
            #        next_x += ddx
            #        next_y += ddy
            #        (free_pos, out_of_map) = self.__isItFreePosition(next_x, next_y)
            #else:
            #    pass
        
        distance_measured = np.sqrt((last_free_pos_x - x)**2 + (last_free_pos_y - y)**2)
        #error = np.random.normal(scale = 0.015 * distance_measured)
        # error = np.random.normal(scale = 0.15 * distance_measured)
        return  distance_measured
    #========================================================================#
    def medicion_esperada(self,x,y,theta,paso):
        # dtheta = 2 * math.pi / self.lidar_resolution
        dtheta = 2 * np.pi / self.lidar_resolution
        dtheta = paso * dtheta
        lidar_angle = 0
        measure_values = np.zeros(int(self.lidar_resolution / paso))
        for i in range(0, int(self.lidar_resolution / paso)):
            measure_values[i] = self.ray_casting(x, y, theta - self.lidar_pos[2] + lidar_angle)
            lidar_angle += dtheta
        return measure_values
    #========================================================================#
    def crear_nube_puntos(self,vector):
        
        N = len(vector)
        nube = np.zeros((2,N))
        
        dtheta = 2 * np.pi / N
        lidar_angle = 0
        
        for i in range(N):
            x_i = vector[i] * np.cos(lidar_angle - self.lidar_pos[2])
            y_i = vector[i] * np.sin(lidar_angle - self.lidar_pos[2])
            nube[0,i] = x_i
            nube[1,i] = y_i
            lidar_angle += dtheta
        return nube

    #========================================================================#
    def closest_point_matching(self,X, P):
      
      P_matched = P.copy()
      
      # cantidad de puntos
      N = X.shape[1]
      orden = np.random.permutation(N)
      for l in range(0,N):
          # elijo un punto de la nube de puntos
          i = orden[l]
          x_i = X[:,i]
          # busco el punto más cercano
          dist_min = np.linalg.norm(x_i - P_matched[:,i])
          k = i
          for m in range(l+1,N):
              j = orden[m]
              dist = np.linalg.norm(x_i - P_matched[:,j])
              if(dist<dist_min):
                  x_aux = X[:,j]
                  p_aux = P_matched[:,i]
                  delta_E = dist - dist_min - np.linalg.norm(x_aux - P_matched[:,j]) + np.linalg.norm(x_aux - p_aux)
                  if(delta_E < 0):
                      dist_min = dist
                      k = j
          aux = P_matched[:,k].copy()
          P_matched[:,k] = P_matched[:,i].copy()
          P_matched[:,i] = aux
              
      return P_matched
    
    #========================================================================#
    def matchear_mediciones(self,nube_h_original,nube_measurements,N,n_particula):

        # creo las nubes de puntos sub muestreadas

        # pose_aux = (self.particles[n_particula]).particle_pos()
        # theta_aux = pose_aux[2]
        # punto_aux = pose_aux[0:2]
        # punto_aux.shape = (2,1)

        e_old = 1000
        for i in range(3):
            
            #calculate RMSE
            e = 0
            for j in range(0,nube_h_original.shape[1]):
                
              e = e+(nube_h_original[0,j]-nube_measurements[0,j])**2 + (nube_h_original[1,j]-nube_measurements[1,j])**2
            
            e = np.sqrt(e/nube_h_original.shape[1])
            #print("error icp: ", e)
            if(abs(e - e_old) < 1e-2):
                break
            
            e_old = e
            
            #nube_h_sub = self.sub_muestrar(nube_h_original,N)
            
            nube_h_original = self.closest_point_matching(nube_measurements,nube_h_original)
            #nube_h_sub = self.closest_point_matching(nube_measurements_sub,nube_h_sub)
            
            #substract center of mass
            mx = np.transpose([np.mean(nube_measurements,1)])
            mp = np.transpose([np.mean(nube_h_original,1)])
            X_prime = nube_measurements-mx
            P_prime = nube_h_original-mp
            
            # mx = np.transpose([np.mean(nube_measurements_sub,1)])
            # mp = np.transpose([np.mean(nube_h_sub,1)])
            # X_prime = nube_measurements_sub-mx
            # P_prime = nube_h_sub-mp
            
            #singular value decomposition
            W = np.dot(X_prime,np.transpose(P_prime))
            U, s, V = np.linalg.svd(W)
    
            #calculate rotation and translation
            R = np.dot(U,np.transpose(V))
            t = mx-np.dot(R,mp)
        
            #apply transformation
            nube_h_original = np.dot(R,nube_h_original)+t

            # print(t.shape,R.shape,punto_aux.shape)

        #     punto_aux = R @ punto_aux + t
        
        # pose_aux = np.array([punto_aux[0][0],punto_aux[1][0],theta_aux])
        # (self.particles[n_particula]).set_particle_pos(pose_aux)

        return nube_h_original
    #========================================================================#
    def resample_particles(self,w):
        #w_max = w[0]
        #pos_max = 0
        # print(w)
        #for i in range(1,len(w)):
        #    if(w[i] > w_max):
        #        w_max = w[i]
        #        pos_max = i
        
        w_max = max(w)
        #w = np.exp(w-w_max)
        # print(w)
        # sumo los pesos
        eta = np.sum(w)
        
        # # normalizo los pesos
        w = np.divide(w,eta)
        
        c = []    
        c.append(w[0])
        
        for i in range(1,self.N):
            c.append(c[i-1] + w[i])
        
        step = 1/(self.N)
        
        seed = np.random.uniform(0,step)
        
        i = 0
        u = seed
        p_sampled = []
        #p_sampled.append(self.particles[i])
        # resample the particles based on the seed , step and cacluated pdf
        for h in range(self.N):
#            '''Write the code here'''
            while u > c[i]:
                i = i + 1
            particula_elegida = self.particles[i]
            #print("elijo partícula: ", i)
            particula_elegida.peso = w[i]
            p_sampled.append(particula_elegida)
            u = u + 1/self.N
        
        self.particles = p_sampled
    #========================================================================#
    def update_particles_measurements(self,measurements):
        
        paso = 8
        
        # vector de pesos weights
        weights = []
        measurements = measurements[0::paso]
        nube_measurements = self.crear_nube_puntos(measurements)
        M = len(measurements)
        for i in range(self.N):
            
            x = (self.particles[i]).x
            y = (self.particles[i]).y
            theta = (self.particles[i]).theta
            
            # busco el vector de medición esperada
            medicion_esperada = self.medicion_esperada(x,y,theta,paso)
            
            # matcheo
            # creo las nubes de puntos a partir de las mediciones
            nube_h_original = self.crear_nube_puntos(medicion_esperada)

            # a partir de ambas nubes de puntos, hago el matcheo de ambas
            nube_h_original = self.matchear_mediciones(nube_h_original,nube_measurements,M,i)
            
            # a partir de la nube de puntos, calculo las distancias
            # h = np.zeros(M)
            # for j in range(M):
            #    h[j] = np.linalg.norm(nube_h_original[:,j])
            #h = medicion_esperada
            
            # diff = h - measurements
            
            # variance = (0.15 * measurements)**2
            
            # prob = np.prod(np.exp(-(diff)**2 / (2 * variance)) / (np.sqrt(variance * 2 * np.pi)))
            #prob = np.sum(-0.5 * np.log(2 * np.pi * variance) + (-(diff)**2) / (2 * variance))
            #print(prob)
            prob = 0

            for j in range(M):
                prob += np.linalg.norm(nube_h_original[:,j] - nube_measurements[:,j])
            
            prob = 1/prob
            weights.append(prob)
            (self.particles)[i].peso = prob
          
            
        self.resample_particles(weights)
        
    #========================================================================#
    # paso de actualización según el modelo de movimiento. Se actualizan las
    # poses de todas las partículas
    def update_particles_motion(self,v,w,t):
        # actualizo la pose de todas las partículas
        
        # actualizo según la odometría
        p_new = []
        for i in range(self.N):
            #(self.particles[i]).mov_odom(v,w,t,self.noise_param[0],self.noise_param[1],self.noise_param[2],self.noise_param[3])
            p_new.append((self.particles[i]).mov_odom(v,w,t,self.noise_param))

        self.particles = p_new 

In [4]:
# Utils: diferentes funciones utilizadas durante la simulación

# Recibe 2 lecturas de odometría y calcula la velocidad lineal y anugular
def motion_model_velocity(od_old,od_new,dt):
    
    v = np.sqrt((od_old[0] - od_new[0])**2 + (od_old[1] - od_new[1])**2) / dt

    delta_theta = od_new[2] - od_old[2]

    while(delta_theta < -0.5 * dt):
        delta_theta += 2*np.pi
    while(delta_theta > 0.5 * dt):
        delta_theta -= 2*np.pi
    
    w = (delta_theta)/dt

    return (v,w)

#========================================================================#  
# Dibuja las partículas en el plot  
def drawparticles(ax, p):
    points_x = np.zeros(len(p))
    points_y = np.zeros(len(p))

    for i in range(0, len(p)):

        points_x[i] = p[i].x
        points_y[i] = p[i].y


    particlepoints = ax.scatter(points_x, points_y, c = 'g',marker = '.')

    return particlepoints
#========================================================================#
# Actualiza las partículas en el plot
def updateparticles(particlepoints, p, map_center, map_scale):
    points_x = np.zeros(len(p))
    points_y = np.zeros(len(p))

    for i in range(0, len(p)):
        points_x[i] = (p[i].x)*map_scale[0] + map_center[0]
        points_y[i] = (p[i].y)*map_scale[1] + map_center[1]

    particlepoints.set_offsets(np.c_[points_x, points_y])

    return particlepoints
#========================================================================#
# Detección de posible colisión durante la localización, es decir, no se
# sabe todavía donde está el robot.

def anti_colision(medidas_colision):
    # se asume que se está yendo hacia adelante

    dist_seguridad = 0.1  #metros
    radio_robot = 0.175 # metros

    prob_colision = False
    N = len(medidas_colision)  # va a valer 144
    puntos_a_tomar = int(N / 2)

    for i in range(puntos_a_tomar):
        aux = medidas_colision[int(N/2) - int(puntos_a_tomar/2) + i]

        if(aux <= radio_robot + dist_seguridad):
            prob_colision = True

    return prob_colision

#========================================================================#  
# Dibuja las trayectoria en el plot
def drawpath(ax, t):
    points_x = np.zeros(len(t))
    points_y = np.zeros(len(t))

    for i in range(0, len(t)):

        points_x[i] = t[i][0]
        points_y[i] = t[i][1]


    pathpoints = ax.scatter(points_x, points_y, c = 'k',marker = '.')

    return pathpoints

#========================================================================#
# def convolucionar_mapa(mapa):
#     mapa_convolucionado = np.zeros(mapa.shape)
#     for y in range(0,mapa.shape[1]):
#         mapa_convolucionado[0,y] = (2/3) * mapa[0,y] + (1/3) * mapa[1,y]
#     for x in range(1,mapa.shape[0]-1):
#         for y in range(0,mapa.shape[1]):
#             mapa_convolucionado[x,y] = (1/4) * mapa[x-1,y] + (1/2) * mapa[x,y] + (1/4) * mapa[x+1,y]
#     for y in range(0,mapa.shape[1]):
#           mapa_convolucionado[mapa.shape[0]-1,y] = (1/3) * mapa[mapa.shape[0]-2,y] + (2/3) * mapa[mapa.shape[0]-1,y]
    
#     return mapa_convolucionado
#========================================================================#
def generar_trayectoria_final(trayectoria,centro_del_mapa,escala):

    # recibe la trayectoria en puntos del mapa y genera una trayectoria de poses
    largo_trayectoria = len(trayectoria)
    trayectoria_final = []

    for i in range(largo_trayectoria-1):
        x = (trayectoria[i][0] - centro_del_mapa[0])/escala[0]
        y = (trayectoria[i][1] - centro_del_mapa[1])/escala[1]

        x_next = (trayectoria[i+1][0] - centro_del_mapa[0])/escala[0]
        y_next = (trayectoria[i+1][1] - centro_del_mapa[1])/escala[1]
        theta = np.arctan2(y_next-y,x_next-x)
        punto = np.array([x,y,theta])
        trayectoria_final.append(punto)
    
    return trayectoria_final
#========================================================================#
def path_following(trayectoria,pose_robot):
    # recibe la trayectoria y la pose del robot. Devuelve comandos de velocidad
    # para corregir al robot
    delta = 2

    Kv = 0.1
    Kw = 0.1
    largo_trayectoria = len(trayectoria)
    # calculo distancia mínima del robot a la trayectoria
    x = pose_robot[0]
    y = pose_robot[1]
    punto_robot = np.array([x,y])
    dist_min = np.linalg.norm(punto_robot - trayectoria[0][0:2])
    dist_min_j = 0
    for j in range(1,largo_trayectoria):
        dist = np.linalg.norm(punto_robot - trayectoria[j][0:2])
        if(dist < dist_min):
            dist_min = dist
            dist_min_j = j
    
    # Tomo como referencia delta puntos después en la trayectoria
    if(dist_min_j + delta > largo_trayectoria-2):
        ref = trayectoria[largo_trayectoria-2]
    else:
        ref = trayectoria[dist_min_j + delta]
    
    #print(ref,pose_robot)
    

    # genero la matriz R
    R = np.array([[np.cos(ref[2]),np.sin(ref[2]),0],[-np.sin(ref[2]),np.cos(ref[2]),0],[0,0,1]])
    delta_x = ref[0] - pose_robot[0]
    delta_y = ref[1] - pose_robot[1]
    delta_theta = ref[2] - pose_robot[2]
    while(delta_theta > np.pi):
        delta_theta -= 2 * np.pi
    while(delta_theta < -np.pi):
        delta_theta += 2 * np.pi
    
    e = R @ np.array([delta_x,delta_y,delta_theta])
    
    # con este error calculo las velocidades
   
    v = np.sqrt(e[0] ** 2 + e[1] ** 2) * Kv
    #vx = e[0] * Kv
    #vy = e[1] * Kv
    #v = (abs(vx) + abs(vy))/2
    w = e[2] * Kw
    # print(e,v,w)
    if(-0.1 < v < 0.1):
        pass
    else:
        v = np.sign(v) * 0.1
    
    if(-0.1 < w < 0.1):
        pass
    else:
        w = np.sign(w) * 0.1

    return (v,w)

#========================================================================#
def angulo_salida_anti_colision(medidas_colision):

    angulos = np.linspace(-np.pi,np.pi,144)
    dist_acum = 0
    acum = 0
    N = len(medidas_colision)  # va a valer 144
    #puntos_a_tomar = int(N / 2)
    #dist_max = medidas_colision[0]

    for j in range(N):
        acum += angulos[j]*medidas_colision[j]
        dist_acum += medidas_colision[j]

    # Tengo que rotar este ángulo
    alfa = acum/dist_acum

    return alfa
#========================================================================#
def salida_anti_colision_trayectoria(trayectoria,pose_robot):

    # Se busca el punto más cercano de la trayectoria.
    x = pose_robot[0]
    y = pose_robot[1]
    punto_robot = np.array([x,y])
    dist_min = np.linalg.norm(punto_robot - trayectoria[0][0:2])
    dist_min_j = 0
    for j in range(1,largo_trayectoria):
        dist = np.linalg.norm(punto_robot - trayectoria[j][0:2])
        if(dist < dist_min):
            dist_min = dist
            dist_min_j = j
    
    # Tomo como referencia delta puntos después en la trayectoria
    if(dist_min_j > largo_trayectoria-2):
        ref = trayectoria[largo_trayectoria-2]
    else:
        ref = trayectoria[dist_min_j]
    
    return ref

#========================================================================#
# def anti_colision_trayectoria(pose_robot,occ_map):
#     # Dada la pose del robot, se fija si hacia donde está yendo, hay peligro de
#     # Colisión

#     puntos = []
#     # -pi/2
#     punto_0 = pose_robot

#     delta = 5
#     p = occ_map[pose_robot[0],pose_robot[1]]
#     edge_cost = np.linalg.norm(parent - child) + p

#     if(p > 0.1):
#         edge_cost = np.inf

#   else:
#       xvalues = np.arange(-delta,delta+1,1)
#       yvalues = np.arange(-delta,delta+1,1)
#       booleano = False
#       for i in range(0,2*delta):
#           if(booleano == False):
#             for j in range(0,2*delta):
#                 x = xvalues[i] + child[0]
#                 y = yvalues[j] + child[1]
#                 if(occ_map[x,y] < 0.1):
#                     pass
#                 else:
#                     edge_cost = np.inf
#                     booleano = True
#                     break



In [5]:
# Funciones realacionadas al planeamiento de trayectorias
# Planeamiento de trayectorias
#========================================================================#
def get_heuristic(cell, goal):
  
  heuristic = 0
  heuristic = np.linalg.norm(cell - goal)
  #heuristic = np.linalg.norm(cell - goal) * 2
  #heuristic = np.linalg.norm(cell - goal) * 5
  #heuristic = np.linalg.norm(cell - goal) * 10
  return heuristic
#========================================================================#
def get_neighborhood(cell, occ_map_shape):

  
  neighbors = []

  
  x = cell[0]
  y = cell[1]
  
  vecino_1 = np.array([x+1,y+1])
  vecino_2 = np.array([x,y+1])
  vecino_3 = np.array([x-1,y+1])
  
  vecino_4 = np.array([x+1,y])
  vecino_5 = np.array([x-1,y])
  
  vecino_6 = np.array([x+1,y-1])
  vecino_7 = np.array([x,y-1])
  vecino_8 = np.array([x-1,y-1])
  
  # neighbors.append(vecino_1)
  # neighbors.append(vecino_2)
  # neighbors.append(vecino_3)
  # neighbors.append(vecino_4)
  # neighbors.append(vecino_5)
  # neighbors.append(vecino_6)
  # neighbors.append(vecino_7)
  # neighbors.append(vecino_8)
  vecinos = []
  vecinos.append(vecino_1)
  vecinos.append(vecino_2)
  vecinos.append(vecino_3)
  vecinos.append(vecino_4)
  vecinos.append(vecino_5)
  vecinos.append(vecino_6)
  vecinos.append(vecino_7)
  vecinos.append(vecino_8)
  
  #chequeo si alguno de los vecinos está fuera del mapa y lo saco de la lista
  #orden = np.random.permutation(8)
  for i in range(0,8):
      #i = orden[l]
      if(((vecinos[i] >= 0).all()) or ((vecinos[i] < occ_map_shape).all())):
          neighbors.append(vecinos[i])
  return neighbors
#========================================================================#
def get_edge_cost(parent, child, occ_map):

  edge_cost = 0
  
  # #dist = np.linalg.norm(parent - child)
  # p = occ_map[child[0],child[1]]
  # #p = occ_map[parent[0],parent[1]]
  # # El robot ocupa cierto espacio, además de un solo punto. Se deberían chequear
  # # puntos alrededor del robot que cubran el espacio

  # if(p > 0.1):
  #     edge_cost = np.inf
  # else:
  #     #edge_cost = p
  #     edge_cost = np.linalg.norm(parent - child) + p
  delta = 6
  p = occ_map[child[0],child[1]]
  edge_cost = np.linalg.norm(parent - child) + p

  if(p > 0.1):
      edge_cost = np.inf

  else:
      xvalues = np.arange(-delta,delta+1,1)
      yvalues = np.arange(-delta,delta+1,1)
      booleano = False
      for i in range(0,2*delta):
          if(booleano == False):
            for j in range(0,2*delta):
                x = xvalues[i] + child[0]
                y = yvalues[j] + child[1]
                if(occ_map[x,y] < 0.1):
                    pass
                else:
                    edge_cost = np.inf
                    booleano = True
                    break
  return edge_cost
#========================================================================#
def run_path_planning(occ_map, start, goal):
    # cost values for each cell, filled incrementally. 
    # Initialize with infinity
    costs = np.ones(occ_map.shape) * np.inf

    # cells that have already been visited
    closed_flags = np.zeros(occ_map.shape)

    predecessors = -np.ones(occ_map.shape + (2,), dtype=np.int)

    # heuristic for A*
    heuristic = np.zeros(occ_map.shape)   # la dejo en 0 para Dijkstra
    # A*
    for x in range(occ_map.shape[0]):
        for y in range(occ_map.shape[1]):
            heuristic[x, y] = get_heuristic(np.array([x, y]), goal)
    
    # start search
    parent = start
    costs[start[0], start[1]] = 0
    # print(parent,goal)
    # loop until goal is found
    while not np.array_equal(parent, goal):
      
        # costs of candidate cells for expansion (i.e. not in the closed list)
        open_costs = np.where(closed_flags==1, np.inf, costs) + heuristic

        # find cell with minimum cost in the open list
        x, y = np.unravel_index(open_costs.argmin(), open_costs.shape)
        # print(x,y)
        # break loop if minimal costs are infinite (no open cells anymore)
        if open_costs[x, y] == np.inf:
            break
      
        # set as parent and put it in closed list
        parent = np.array([x, y])
        closed_flags[x, y] = 1;
      
        # update costs and predecessor for neighbors
        '''your code here'''
        '''***        ***'''
        # obtengo los nodos adyacentes del nodo más promisorio, es decir, el de 
        # menor costo. "neighbors" es una lista que cada elemento es un vecino del
        # nodo "parent".
        neighbors = get_neighborhood(parent, occ_map.shape)
      
        # calculo los costos para ir desde el nodo "parent" a todos los otros nodos
        # vecinos    
      
        cantidad_de_vecinos = len(neighbors)
      
        for i in range(0,cantidad_de_vecinos):
            # costo parent -> neighbors[i]
            costo_parent_vecino = get_edge_cost(parent,neighbors[i],occ_map)
            # Me fijo si es menos costoso hacer:
            # start -> parent -> neighbors[i] que
            # start -> neighbors[i]
            costo_start_parent_vecino = costs[x,y] + costo_parent_vecino
            #costo_start_parent_vecino = get_edge_cost(start,neighbors[i],occ_map)
            costo_start_vecino = costs[neighbors[i][0],neighbors[i][1]]
            #costo_start_vecino = get_edge_cost(start,neighbors[i],occ_map)
            
            if(costo_start_parent_vecino < costo_start_vecino):
              costs[neighbors[i][0],neighbors[i][1]] = costo_start_parent_vecino.copy()
              predecessors[neighbors[i][0],neighbors[i][1]] = parent.copy()

    # Rewind the path from goal to start (at start predecessor is [-1,-1])
    camino = []

    if np.array_equal(parent, goal):
        path_length = 0
        while predecessors[parent[0], parent[1]][0] >= 0:
            #plot_path(parent, goal)
            #plot_path(parent, goal)
            predecessor = predecessors[parent[0], parent[1]]
            path_length += np.linalg.norm(parent - predecessor)
            parent = predecessor
            camino.insert(0,predecessor)
        print ("found goal     : " + str(parent))
        print ("cells expanded : " + str(np.count_nonzero(closed_flags)))
        print ("path cost      : " + str(costs[goal[0], goal[1]]))
        print ("path length    : " + str(path_length))
    else:
        print ("no valid path found")

    return camino

In [6]:
# Desafio patrullaje

# ##################################################################################3

# Pose inicial REAL del robot (ground truth)
x0 = 0
y0 = 0
theta0 = 0

# Creo una instancia de DiffRobot
my_robot = DiffRobot(x0,y0,theta0)

# Cargo el mapa de grilla de ocupación
# Como resolción (metros/píxeles) uso el valor 5 metros/126 píxeles

occupation_map_file = "imagen_2021_mapa_tp.png"
occupation_map_scale_factor = 5/126

my_robot.loadOccupationMap(occupation_map_file, occupation_map_scale_factor)

# se guardan estas variables para hacer los gráficos del mapa y de la ubicación del robot
(occ_map_height, occ_map_width) = my_robot.getOccMapSizeInWorld()
occ_map = my_robot.getOccMap()

########################################################################################
# Se crea el gráfico donde se va a plottear la simulación
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.pcolormesh(np.flipud(occ_map))
ax.axis('off')
status_text = ax.text(0, -5, "")

map_center = (occ_map.shape[0] /2.0, occ_map.shape[1] / 2.0)
map_scale = (occ_map.shape[0] / occ_map_width, occ_map.shape[1] / occ_map_height)

plt.ion()
(gt_icon, gt_bearing) = drawRobot(ax, 0 + map_center[0], 0 + map_center[1], 0, my_robot.diameter * map_scale[0], 'g')
(od_icon, od_bearing) = drawRobot(ax, 0 + map_center[0], 0 + map_center[1], 0, my_robot.diameter * map_scale[0], 'grey')

plt.draw()
plt.pause(1)

lidar_values = np.zeros(my_robot.getLidarResolution())
lidarPoints = drawLidarMeasure(ax, [0 ,0 ,0], my_robot.getLidarResolution(), lidar_values)

#############################################################################################

# Se configura al robot y comienza la simulacón
my_robot.setLinearVelocity(0.0)
my_robot.setAngularVelocity(0)
my_robot.enableBumper()
my_robot.turnLidar_ON()


# variable donde guardo las lecturas del kalman filter para comparar con ground
# truth, luego de finalizada la simulación
valores_pf = []
valores_reales = []
valores_odometry = []

my_robot.startSimulation(print_status=False)

# Cantidad de iteraciones de la simulación
simulation_steps = 500

# Flags varios #################################################################

# Flags de localización-----
localization_on = True     # Si vale True ==> QUIERO LOCALIZACIÓN
doing_localization = False # Si vale True ==> YA VENÍA HACIENDO LA LOCALIZACIÓN
robot_localizado = False # Esta variable indica si se localizó al robot o no

# Flags de colisión-----
anti_colision_mode = True
posible_colision = False
rutina_colision = False

# Flags de planeamiento de trayectoria-----
trayectoria_planeada = False # Indica si se está ejecutando la rutina de 
                             # planeamiento de trayectorias.

# Flags de seguimiento de trayectoria-----
llegue_objetivo = False # Esta variable indica si llegué al objetivo o no
siguiendo_la_trayectoria = False # Si empecé a seguir la trayectoria o no

# Puntos objetivo del desafío, en metros. Se le resta 2,5 por ser el centro del
# mapa.
objetivos = [np.array([1.82-2.5,1.5-2.5]),np.array([3.14-2.5,3.3-2.5])]
n_objetivo = 0

# Parámetros de la localización ################################################

v_loc = 0.05 # Velocidad lineal durante la localización
w_loc = 0.0 # Velocidad Angular durante la localización

# Parámetros del filtro de partículas-----
error_parameters = np.array([0.10, 0.20, 0.10, 0.20, 0.25, 0.25])
N = 120 # Número de partículas

# Comienza el ciclo ############################################################

for i in range(simulation_steps):

# 1: Se chequea la localización ===============================================# 
    if(localization_on == True):

        # Si no se había empezado con la localización, se la configura por 
        # primera vez.

        if(doing_localization == False):  

            # Inicializo el filtro de partículas
            filtro_de_particulas = particle_filter(N,error_parameters,my_robot.getLidarResolution(),np.array([0.09, 0.0, math.pi]))
            
            # le cargo el mapa
            filtro_de_particulas.loadoccmap(occ_map,occupation_map_scale_factor)

            # Indico que se está ejecutando una rutina de localización
            doing_localization = True

            # Dibujo las partículas en el mapa
            particlepoints = drawparticles(ax,filtro_de_particulas.particles)

            # instantes de tiempo de la simulación lectura de odometría del
            # robot. Se usarán para calcular después las predicciones y 
            # correcciones.
            t_old = my_robot.getSimulationTime()

            # lectura de odometría
            od_old = my_robot.getOdometry()

        else:
            # Si se está ejecutando una rutina de localización, se continua con esta            
            # consulto el tiempo de simulación actual para calcular el paso de
            # tiempo desde la última lectura de odometría
            t_new = my_robot.getSimulationTime()
            T = t_new - t_old
            od_new = my_robot.getOdometry()
            if(T!=0):
                (v,w) = motion_model_velocity(od_old,od_new,T)
                # actualizo las partículas
                filtro_de_particulas.update_particles_motion(v,w,T)

            # Hago una medición
            medidas = my_robot.getLidarMeaurement()

            # Lecturas de odometría y de tiempo, pasadas
            od_old = my_robot.getOdometry()
            t_old = my_robot.getSimulationTime()

            if(medidas.all() != 0):
                filtro_de_particulas.update_particles_measurements(medidas)

            # Me pregunto si el robot cumple la condición de localizado
            desvio = filtro_de_particulas.std_err_particle_pos()
            if(desvio < 0.2):
                robot_localizado = True
                print("El robot está localizado")
            else:
                robot_localizado = False

    else:
        # Se detiene la localización
        doing_localization = False

# 2: Anti colisión y seteo de velocidades =====================================# 
    # Se chequea si hay una posibilidad de chocar. Dependiendo de si estoy
    # siguiendo la trayectoria o no, la anti colisión es diferente.

    if(trayectoria_planeada == True):
        my_robot.setLinearVelocity(v_tray)
        my_robot.setAngularVelocity(w_tray)
        # if(rutina_colision == False):
        #     # RUTINA DURANTE EL SEGUIMIENTO DE TRAYECTORIA
        #     pose_robot = filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()].particle_pos()[0:2]
        #     pose_robot = pose_robot / occupation_map_scale_factor + centro_del_mapa
        #     posible_colision = anti_colision_trayectoria(pose_robot,(np.transpose(np.flipud(1-occ_map))))
        #     if(posible_colision == True):
        #         # Aviso que estoy en rutina de colisión
        #         rutina_colision = True

        #         # Detengo al robot para evitar la colisión
        #         my_robot.setLinearVelocity(0.0)
        #         my_robot.setAngularVelocity(0.0)

        #         pose_robot = filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()].particle_pos()

        #         punto_trayectoria_mas_cercano = salida_anti_colision_trayectoria(trayectoria_final,pose_robot)

        #         # Seteo la velocidad angular
        #         alfa = np.arctan2(punto_trayectoria_mas_cercano[1]-pose_robot[1],punto_trayectoria_mas_cercano[0] - pose_robot[0])

        #         delta_theta_trayectoria = alfa - my_robot.getOdometry[2]

        #         if(delta_theta_trayectoria > 0):
        #             my_robot.setAngularVelocity(0.1)
        #         else:
        #             my_robot.setAngularVelocity(-0.1)

        #         # Me guardo el ángulo de rotación actual, para después chequear
        #         # si ya roté
        #         theta_old = my_robot.getOdometry()[2]

        # else:
        #     # Me fijo si ya roté
        #     if(abs(my_robot.getOdometry()[2] - theta_old) >= abs(alfa)):
        #         # Si roté, aviso que salgo de la rutina de colisión y detengo
        #         # al robot
        #         rutina_colision = False
        #         my_robot.setLinearVelocity(0.0)
        #         my_robot.setAngularVelocity(0.0)

    else:
        # RUTINA DURANTE LOCALIZACIÓN
        if(rutina_colision == False):
            medidas_colision = my_robot.getLidarMeaurement()
            if(medidas_colision.all() != 0):
                posible_colision = anti_colision(medidas_colision)
            else:
                posible_colision = False

            if(posible_colision == True):
                # Aviso que estoy en rutina de colisión
                rutina_colision = True

                # Detengo al robot para evitar la colisión
                my_robot.setLinearVelocity(0.0)
                my_robot.setAngularVelocity(0.0)

                # Calculo un nuevo ángulo de rotación, para "escapar" de la 
                # colisión
                alfa = angulo_salida_anti_colision(medidas_colision)

                # Seteo la velocidad angular

                if(alfa > 0):
                    my_robot.setAngularVelocity(0.1)
                else:
                    my_robot.setAngularVelocity(-0.1)

                # Me guardo el ángulo de rotación actual, para después chequear
                # si ya roté
                theta_old = my_robot.getOdometry()[2]

            else:
                # Si no hay posibilidades de colisión, se setean las velocidades 
                # del robot según lo que se requiera para la localización.
                my_robot.setLinearVelocity(v_loc)
                my_robot.setAngularVelocity(w_loc)

        # Si ya venía con la rutina de colisión, sigo rotando
        else:
            # Me fijo si ya roté
            if(abs(my_robot.getOdometry()[2] - theta_old) >= abs(alfa)):
                # Si roté, aviso que salgo de la rutina de colisión y detengo
                # al robot
                rutina_colision = False
                my_robot.setLinearVelocity(0.0)
                my_robot.setAngularVelocity(0.0)

# 3: Planeamiento de trayectorias =============================================# 
    # El planeamiento de la trayectoria no se realiza hasta que se el localizador
    # conozca la pose del robot en el mapa.

    # Me fijo si ya se planee la trayectoria o no, para no volver a planearla de
    # nuevo.
    if(trayectoria_planeada == False):

        if((robot_localizado == False)):
            # Si el robot no está localizado, no hago nada
            pass
    
        else:
            # Si el robot sí está localizado, comienzo el planeamiento de la
            # trayectoria

            # Punto inicial: posición del robot (según el filtro de partículas)
            start = filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()].particle_pos()[0:2]

            centro_del_mapa = np.array(map_center)
            escala = np.array(map_scale)
            
            # Objetivo de la trayectoria
            goal = objetivos[n_objetivo]

            # Se escalan ambos puntos a pixeles
            start = start /  occupation_map_scale_factor + centro_del_mapa

            goal = goal / occupation_map_scale_factor + centro_del_mapa

            start = np.array([floor(start[0]),floor(start[1])])
            goal = np.array([floor(goal[0]),floor(goal[1])])
            
            # Devuevlo la trayectoria a seguir
            trayectoria = run_path_planning((np.transpose(np.flipud(1-occ_map))), start, goal)
            
            # Convierto la trayectoria en poses en el mapa
            trayectoria_final = generar_trayectoria_final(trayectoria,centro_del_mapa,escala)

            # Dibujo la traycetoria
            drawpath(ax, trayectoria)

            # Indico que ya planeé la trayectoria
            trayectoria_planeada = True
            llegue_objetivo = False
            siguiendo_la_trayectoria = False

            # Se detiene al robot.
            my_robot.setLinearVelocity(0.0)
            my_robot.setAngularVelocity(0.0)

# 4: Seguimiento de trayectorias ==============================================# 
    if(trayectoria_planeada == True):
        # Si ya planee la trayectoria, debo seguirla

        # Me pregunto si llegué al objetivo
        pose_robot = filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()].particle_pos()[0:2]
        
        objetivo = trayectoria_final[len(trayectoria_final)-1][0:2]
        
        dist_objetivo = np.linalg.norm(pose_robot - objetivo)
        
        # Si llegué levanto el flag de objetivo alcanzado
        if(dist_objetivo < 0.1):
            llegue_objetivo = True

        # Si no llegué, sigo recorriendo la trayectoria   
        if(llegue_objetivo != True):
            
            pose_robot = filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()].particle_pos()
            (v_tray,w_tray) = path_following(trayectoria_final,pose_robot)

            if(siguiendo_la_trayectoria == False):
                v_tray = 0
                pose_robot = filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()].particle_pos()
                theta_trayectoria = pose_robot[2]-trayectoria_final[0][2]
                if(abs(theta_trayectoria) < 0.2):
                    siguiendo_la_trayectoria = True

        # Si llegué, me freno y aviso que llegué
        else:
            my_robot.setLinearVelocity(0.0)
            my_robot.setAngularVelocity(0.0)
            print("Objetivo alcanzado: ", filtro_de_particulas.particles[filtro_de_particulas.best_particle_pos()].particle_pos())
            llegue_objetivo = False
            if(n_objetivo == 0):
                n_objetivo = 1
                trayectoria_planeada = False

            else:
                # Si ya llegué a los 2 objetivos, salgo para terminar el
                # programa.
                break
                   
# 4: Ploteo de la simulación ==================================================# 
    best_particle = filtro_de_particulas.best_particle_pos()
    
    print("GT: ", my_robot.getGroundTruth(),"PF: ", filtro_de_particulas.particles[best_particle].particle_pos(), "t: ", my_robot.getSimulationTime())

    valores_pf.append(filtro_de_particulas.particles[best_particle].particle_pos())
    valores_reales.append(my_robot.getGroundTruth())
    valores_odometry.append(my_robot.getOdometry())
    
    # The state of the robot is consulted for drawing purpose
    # To update de draw of robot's status could take more time that
    # simulation step (dt = 0.1 seg.)
    # As simulation run in an independent thread the draw will be refreshing 
    # at a lower frequency.
    od = my_robot.getOdometry()
    gt = my_robot.getGroundTruth()

    gt_icon.center = gt[0] * map_scale[0] + map_center[0], gt[1] * map_scale[1] + map_center[1]
    gt_bearing.set_xdata([gt[0] * map_scale[0] + map_center[0], gt[0] * map_scale[0] + map_center[0] + 0.5 * map_scale[0] * my_robot.diameter * np.cos(gt[2])])
    gt_bearing.set_ydata([gt[1] * map_scale[1] + map_center[1], gt[1] * map_scale[1] + map_center[1] + 0.5 * map_scale[1] * my_robot.diameter * np.sin(gt[2])])
    
    od_icon.center = od[0]* map_scale[0]  + map_center[0], od[1] * map_scale[1]  + map_center[1]
    od_bearing.set_xdata([od[0] * map_scale[0] + map_center[0], od[0] * map_scale[0] + map_center[0] + 0.5 * my_robot.diameter * map_scale[0] * np.cos(od[2])])
    od_bearing.set_ydata([od[1] * map_scale[1] + map_center[1], od[1] * map_scale[1] + map_center[1] + 0.5 * my_robot.diameter * map_scale[1] * np.sin(od[2])])
    
    lidar_values = my_robot.getLidarMeaurement()
    updatedLidarDraw(lidarPoints, gt, lidar_values, my_robot.getLidarResolution(), map_center, map_scale)

    updateparticles(particlepoints, filtro_de_particulas.particles, map_center, map_scale)

    v = my_robot.getLinearVelocity()
    w = my_robot.getAngularVelocity()
    s_text = "time = " + "{:.2f}".format(my_robot.getSimulationTime()) + "seg   u_t=(" + "{:.2f}".format(v)  + " ; " + "{:.2f}".format(w) + ") Collision = " + str(my_robot.getBumperSensorValue()) 
    status_text.set_text(s_text)

    plt.draw()
    file = "/content/plots/_" + str(i).zfill(3)
    plt.savefig(file)
    plt.pause(my_robot.dt)

    # As plot function takes time it is not needed to sleep main thread
    # if it is not the case consider to sleep main thread.
    time.sleep(my_robot.dt)

my_robot.stopSimulation()


<IPython.core.display.Javascript object>

GT:  [0. 0. 0.] PF:  [-0.09367567 -1.02953123 -2.30609222] t:  0.0
GT:  [ 1.04045387e-01  4.84690462e-05 -2.17802831e-05] PF:  [0.12031223 0.03757092 0.08016846] t:  2.1
GT:  [ 0.10182174  0.00043904 -0.2241584 ] PF:  [0.19402783 0.04037905 0.03807564] t:  4.3
GT:  [ 0.10182174  0.00043904 -0.2241584 ] PF:  [-0.14535617  0.03020332 -0.08326713] t:  6.800000000000001
GT:  [ 0.10381579 -0.00049035 -0.47108128] PF:  [ 0.26598158  0.01627804 -0.3936437 ] t:  9.200000000000001
GT:  [ 0.10381579 -0.00049035 -0.47108128] PF:  [ 0.22111095  0.03416883 -0.41416109] t:  11.700000000000001
GT:  [ 0.21886831 -0.05915737 -0.4720284 ] PF:  [ 0.23852711  0.02649898 -0.41483085] t:  14.3
GT:  [ 0.32947695 -0.11528094 -0.46848173] PF:  [ 0.15490585 -0.1292908  -0.50973781] t:  16.8
GT:  [ 0.4372881  -0.16973688 -0.46743086] PF:  [ 0.38140339 -0.26322953 -0.54246787] t:  19.200000000000003
GT:  [ 0.54952889 -0.22640469 -0.46695986] PF:  [ 0.37990018 -0.16050618 -0.36730647] t:  21.700000000000003
GT:  [

In [ ]:
  my_robot.stopSimulation()

finishing simulation thread...
Simulation Thread Stopped


In [ ]:
#punto = trayectoria[0:6]
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.pcolormesh(np.flipud(occ_map))
ax.axis('on')
drawpath(ax, trayectoria)
plt.savefig("/content/plots/mapa_escalado")
#print(1-np.flipud(occ_map)[63,47])

In [ ]:
%matplotlib notebook

In [8]:
# cargar datos
from google.colab import files
uploaded = files.upload()

In [ ]:
# Ejecutar esta celda para descargar los gráficos de la simulación
!zip -r /content/plots.zip /content/plots
files.download("/content/plots.zip")